In [ ]:
import os, pandas as pd, networkx as nx, numpy as np, statsmodels.api as sm
from sqlalchemy import create_engine
from datetime import date, datetime, timedelta
import matplotlib.pyplot as plt, seaborn as sns

## parameters

In [ ]:
yeari, yearf = '2024', '2024'
weeki, weekf = '18', '31'

In [ ]:
di = datetime.strptime(f'{yeari}-{weeki}-1', "%Y-%W-%w").date()
df = datetime.strptime(f'{yearf}-{weekf}-1', "%Y-%W-%w").date() + timedelta(6)
ds = [di+timedelta(dt) for dt in range((df-di).days+1)]
daylist = ds
print(di, 'until', df)

## load data

In [ ]:
# load stadium meta data
stadname_data = pd.read_csv('output/00_stadname_data.csv')
aid2name = {aid: name for aid, name in zip(stadname_data.area_id, stadname_data.area_name)}

aid2city = {aid: city for aid, city in sorted({
    1: 'München',
    2: 'Berlin',
    3: 'Leipzig',
    4: 'Frankfurt am Main',
    10: 'Dortmund',
    11: 'Stuttgart',
    15: 'Köln',
    24: 'Düsseldorf',
    25: 'Hamburg',
    28: 'Gelsenkirchen',

    16: 'Berlin',
    29: 'Hamburg',
    41: 'München',
    64: 'Frankfurt am Main',
}.items(), key=lambda item: item[1])}
city2aid = {city: aid for aid, city in [(aid, city) for aid, city in aid2city.items()][::-1]}

In [ ]:
# n_d stadiums: load did numbers for stadiums
panelstad_data2 = pd.read_csv('output/00_panelstad_data2.csv')
panelstad_data2['day'] = [d.date() for d in pd.to_datetime(panelstad_data2.day)]

# load mass event data
match_data = pd.read_csv('output/00_event_data.csv')
match_data['day'] = [d.date() for d in pd.to_datetime(match_data.day)]

## plot stadium $n_d$

In [ ]:
sns.set_theme(style="ticks")

# Define the palette as a list to specify exact values
#palette = sns.husl_palette(2)#sns.color_palette("rocket_r")

# Plot the lines on two facets
g = sns.relplot(
    data=panelstad_data2,
    x="day", y="ndids",
    #hue="with_stadiums", hue_order=[True,False],
    row="area_id", row_order=list(aid2city.keys()),# size="choice",# col="align",
    kind="line",# palette=palette,# size_order=[False, True], 
    height=2, aspect=4., facet_kws=dict(sharex=False),
)

axes = g.axes
holis = [date(2024,5,9), date(2024,5,20)]
for ax_row, (aid, city) in zip(axes, aid2city.items()):
    for ax in ax_row:
        ax.set_ylabel('users in stadium')
        ax.set_title(f"{city.split(' ')[0]} - {aid2name[aid]}")
        #daylist = [di+timedelta(dt) for dt in range(1+(df-di).days)]
        ax.set_xticks(daylist)
        ax.set_xticklabels([str(d.month).zfill(2)+'/'+str(d.day).zfill(2) if d.weekday()==6 else '' for t,d in enumerate(daylist)])#, rotation=90)
        #ax.set_xlim([di-timedelta(7), df+timedelta(7)])
        ymax = 150.# 90.
        ax.set_ylim([0., ymax])
        matches_here = match_data[match_data.area_id==aid]
        if city=='Gelsenkirchen':
            matches_here = pd.concat([matches_here, match_data[match_data.match=='Schalke Tach']])
        for day, match in zip(matches_here.day, matches_here.match):
            if day in daylist:
                ax.text(day, ymax, match, rotation=90, ha='center', va='top', fontsize=10)

plt.savefig(f'plots/01_panelstad.jpg', bbox_inches='tight', dpi=300)
plt.savefig(f'plots/01_panelstad.pdf', bbox_inches='tight')
plt.show()

In [ ]:
sns.set_theme(style="ticks")

# Define the palette as a list to specify exact values
#palette = sns.husl_palette(2)#sns.color_palette("rocket_r")

# Plot the lines on two facets
g = sns.relplot(
    data=panelstad_data2[panelstad_data2.area_id.isin(list(aid2city.keys())[:7])],
    x="day", y="ndids",
    #hue="with_stadiums", hue_order=[True,False],
    row="area_id", row_order=list(aid2city.keys())[:7],# size="choice",# col="align",
    kind="line",# palette=palette,# size_order=[False, True], 
    height=2, aspect=4., facet_kws=dict(sharex=False),
)

axes = g.axes
holis = [date(2024,5,9), date(2024,5,20)]
for ax_row, (aid, city) in zip(axes, [(aid, city) for aid, city in aid2city.items()][:7]):
    for ax in ax_row:
        ax.set_ylabel('users in stadium')
        ax.set_title(f"{city.split(' ')[0]} - {aid2name[aid]}")
        #daylist = [di+timedelta(dt) for dt in range(1+(df-di).days)]
        ax.set_xticks(daylist)
        ax.set_xticklabels([str(d.month).zfill(2)+'/'+str(d.day).zfill(2) if d.weekday()==6 else '' for t,d in enumerate(daylist)])#, rotation=90)
        #ax.set_xlim([di-timedelta(7), df+timedelta(7)])
        ymax = 150.# 90.
        ax.set_ylim([0., ymax])
        matches_here = match_data[match_data.area_id==aid]
        if city=='Gelsenkirchen':
            matches_here = pd.concat([matches_here, match_data[match_data.match=='Schalke Tach']])
        for day, match in zip(matches_here.day, matches_here.match):
            if day in daylist:
                ax.text(day, ymax, match, rotation=90, ha='center', va='top', fontsize=10)            

plt.savefig(f'plots/01_panelstad_1.jpg', bbox_inches='tight', dpi=300)
plt.savefig(f'plots/01_panelstad_1.pdf', bbox_inches='tight')
plt.show()

In [ ]:
sns.set_theme(style="ticks")

# Define the palette as a list to specify exact values
#palette = sns.husl_palette(2)#sns.color_palette("rocket_r")

# Plot the lines on two facets
g = sns.relplot(
    data=panelstad_data2[panelstad_data2.area_id.isin(list(aid2city.keys())[7:])],
    x="day", y="ndids",
    #hue="with_stadiums", hue_order=[True,False],
    row="area_id", row_order=list(aid2city.keys())[7:],# size="choice",# col="align",
    kind="line",# palette=palette,# size_order=[False, True], 
    height=2, aspect=4., facet_kws=dict(sharex=False),
)

axes = g.axes
holis = [date(2024,5,9), date(2024,5,20)]
for ax_row, (aid, city) in zip(axes, [(aid, city) for aid, city in aid2city.items()][7:]):
    for ax in ax_row:
        ax.set_ylabel('users in stadium')
        ax.set_title(f"{city.split(' ')[0]} - {aid2name[aid]}")
        #daylist = [di+timedelta(dt) for dt in range(1+(df-di).days)]
        ax.set_xticks(daylist)
        ax.set_xticklabels([str(d.month).zfill(2)+'/'+str(d.day).zfill(2) if d.weekday()==6 else '' for t,d in enumerate(daylist)])#, rotation=90)
        #ax.set_xlim([di-timedelta(7), df+timedelta(7)])
        ymax = 125.# 90.
        ax.set_ylim([0., ymax])
        matches_here = match_data[match_data.area_id==aid]
        for day, match in zip(matches_here.day, matches_here.match):
            if day in daylist:
                ax.text(day, ymax, match, rotation=90, ha='center', va='top', fontsize=10)

plt.savefig(f'plots/01_panelstad_2.jpg', bbox_inches='tight', dpi=300)
plt.savefig(f'plots/01_panelstad_2.pdf', bbox_inches='tight')
plt.show()

In [ ]:
aid = 11# 28:Gelsenkirchen, 11:Stuttgart
city = aid2city[aid]
sns.set_theme(style="ticks")

# Define the palette as a list to specify exact values
#palette = sns.husl_palette(2)#sns.color_palette("rocket_r")

# Plot the lines on two facets
g = sns.relplot(
    data=panelstad_data2[panelstad_data2.area_id==aid],
    x="day", y="ndids",
    #hue="with_stadiums", hue_order=[True,False],
    #row="area_id", row_order=list(aid2city.keys())[:7],# size="choice",# col="align",
    kind="line",# palette=palette,# size_order=[False, True], 
    height=2, aspect=5., facet_kws=dict(sharex=False),
)

ax = g.axes[0][0]
holis = [date(2024,5,9), date(2024,5,20)]
ax.set_ylabel('users in stadium')
ax.set_title(f"{city.split(' ')[0]} - {aid2name[aid]}")
#daylist = [di+timedelta(dt) for dt in range(1+(df-di).days)]
ax.set_xticks(daylist)
ax.set_xticklabels([str(d.month).zfill(2)+'/'+str(d.day).zfill(2) if d.weekday()==6 else '' for t,d in enumerate(daylist)])#, rotation=90)
ax.set_yticks([0,50,100,150])
#ax.yaxis.grid(True)
#ax.set_xlim([di-timedelta(7), df+timedelta(7)])
ymax = 150.# 90.
ax.set_ylim([-2.5, ymax])
matches_here = match_data[match_data.area_id==aid]
if city=='Gelsenkirchen':
    matches_here = pd.concat([matches_here, match_data[match_data.match=='Schalke Tach']])
#for day, match in zip(matches_here.day, matches_here.match):
#    if day in daylist:
#        ax.text(day, ymax+100, match, rotation=90, ha='center', va='top')#, fontsize=10)            

plt.savefig(f'plots/01_panelstad_single.jpg', bbox_inches='tight', dpi=300)
plt.savefig(f'plots/01_panelstad_single.pdf', bbox_inches='tight')
plt.show()